<a href="https://colab.research.google.com/github/Galahexolion/Giliojo-mokymo-sistem-taikymai/blob/main/Lab8_LSTM/Lab8_LSTM_Weather_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- 1. DUOMENŲ ĮKĖLIMAS ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

# Įsitikinkite, kad lab_data.csv yra įkeltas į Colab
try:
    df = pd.read_csv('lab_data.csv')
    print("Duomenys įkelti iš failo.")
except:
    # Jei failo nėra, siunčiamės iš GitHub (kaip numatyta lab faile)
    url = "https://raw.githubusercontent.com/vytkuc/inf4039_2024_autumn/refs/heads/main/lab06_RNN/lab_data.csv"
    df = pd.read_csv(url)
    print("Duomenys atsisiųsti iš interneto.")

# Vizualizacija
def plot_timeseries(date, value, title="Median Daily Temperatures"):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=date, y=value, mode='lines', name='Value', line=dict(color='black', width=1)))
    fig.update_layout(title=title, plot_bgcolor='white',
                      xaxis=dict(showline=True, showgrid=True, linecolor='black'),
                      yaxis=dict(showline=True, showgrid=True, linecolor='black'))
    fig.show()

plot_timeseries(df['Date'], df['MedTemp'])

Duomenys įkelti iš failo.


In [2]:
# --- 2. DUOMENŲ PARUOŠIMAS ---

# Normalizavimas
scaler = MinMaxScaler()
data_values = df['MedTemp'].values.reshape(-1, 1)
X_scaled = scaler.fit_transform(data_values)

# Train/Test atskyrimas (nešufliuojant!)
train_data, test_data = train_test_split(X_scaled, test_size=0.2, shuffle=False)

# Sekų formavimas (Time Steps)
def get_XY(dat, time_steps):
    # Yra niuansas su indeksais, darome paprastą slenkančio lango funkciją
    X, Y = [], []
    for i in range(len(dat) - time_steps):
        X.append(dat[i:(i + time_steps), 0])
        Y.append(dat[i + time_steps, 0])
    return np.array(X).reshape(-1, time_steps, 1), np.array(Y)

time_steps = 7
trainX, trainY = get_XY(train_data, time_steps)
testX, testY = get_XY(test_data, time_steps)

print(f"Train X shape: {trainX.shape}, Train Y shape: {trainY.shape}")

Train X shape: (2735, 7, 1), Train Y shape: (2735,)


In [3]:
# --- 3. LSTM MODELIS ---

model = keras.Sequential([
    layers.Input(shape=(time_steps, 1)),
    # LSTM sluoksnis vietoje SimpleRNN
    # activation='relu' kartais naudojamas, bet standartas yra 'tanh' (greičiau veikia su GPU)
    layers.LSTM(50, activation='tanh', return_sequences=False),
    layers.Dense(1, activation='linear')
])

model.summary()

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Mokymas
history = model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=1, shuffle=False)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0178 - mae: 0.0928
Epoch 2/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0109 - mae: 0.0766
Epoch 3/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0082 - mae: 0.0680
Epoch 4/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0077 - mae: 0.0664
Epoch 5/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0076 - mae: 0.0661
Epoch 6/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0074 - mae: 0.0657
Epoch 7/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0073 - mae: 0.0652
Epoch 8/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0071 - mae: 0.0647
Epoch 9/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.0070 - mae: 0.0645
Epoch 10/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0069 - mae: 0.0642
Epoch 11/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0068 - mae: 0.0640
Epoch 12/20
2735/2735 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0068 - mae: 0.0637

In [4]:
# --- 4. VERTINIMAS ---

# Prognozės
pred_train = model.predict(trainX)
pred_test = model.predict(testX)

# Atstatome skales
pred_test_inv = scaler.inverse_transform(pred_test)
y_test_inv = scaler.inverse_transform(testY.reshape(-1, 1))

print(f"\nTest MSE: {mean_squared_error(y_test_inv, pred_test_inv):.4f}")

# Grafikas
def plot_predicted(true_data, pred_data):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=true_data.flatten(), mode='lines', name='Actual', line=dict(color='black', width=1)))
    fig.add_trace(go.Scatter(y=pred_data.flatten(), mode='lines', name='Predicted (LSTM)', line=dict(color='red', width=1)))
    fig.update_layout(title="LSTM Predictions vs Actual", plot_bgcolor='white')
    fig.show()

plot_predicted(y_test_inv, pred_test_inv)

86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

Test MSE: 16.1841
